In [2]:
%pip install joblib

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install "dask[complete]"
%pip install dask_ml

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [16]:
from joblib import Parallel, delayed
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select   # seleccion de un dropdown
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import pandas as pd
import multiprocessing as mp
import warnings
warnings.filterwarnings('ignore')

from joblib._parallel_backends import LokyBackend
import asyncio

from tqdm.notebook import tqdm
import dask
import dask.dataframe as dd

In [5]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [17]:
meses = ["january","february","march","april","may","june","july","august","september","october"]
dias = []
for i in range(1,32):
    dias.append(str(i))


In [18]:
urls = []
for mes in meses:
    for dia in dias:
        urls.append(f'https://en.tutiempo.net/records/lemd/{dia}-{mes}-2023.html')

In [102]:
urls[279]

'https://en.tutiempo.net/records/lemd/1-october-2023.html'

In [75]:
def extraer(url):
    table = []  # Inicializa table como una lista vacía
    columns = []
    
    try:
        # inicia el driver
        driver = webdriver.Chrome()
        driver.get(url)

        time.sleep(2)
        # acepta cookies
        aceptar = driver.find_element(By.XPATH,'/html/body/div[18]/div[2]/div[1]/div[2]/div[2]/button[1]')
        aceptar.click()

        time.sleep(2)
        #aceptar = WebDriverWait(driver, 10).until(driver.find_element(By.XPATH, '//*[@id="DivAceptarCookies"]/div/a[2]'))
        aceptar = driver.find_element(By.XPATH, '//*[@id="DivAceptarCookies"]/div/a[2]')
        aceptar.click()
        
        time.sleep(2)
        day = driver.find_elements(By.XPATH, '//table//tbody//tr')[1].text
        table = [row.text.split('\n')[0:3] + row.text.replace(' km/h', '').split('\n')[-1].split(' ', 2)
                 for row in driver.find_elements(By.XPATH, '//table//tbody//tr')[3::2]]

        columns = ["Day", "Hour", "Condition", "Temperature", "Wind", "Relative_hum", "Pressure"]

        for i in table:
            i.insert(0, day)
            
        return pd.DataFrame(table, columns=columns)
        
    except Exception as e:  # Captura cualquier excepción y muestra el mensaje de error
        print(f"Error: {e}")
        print(f"Error en la URL: {url}")
        # En caso de error, simplemente se devolverá la lista vacía 'table'
        return pd.DataFrame(table, columns=columns) 
    driver.quit()
#     return pd.DataFrame(table, columns=columns)

In [107]:
%%time
paralelo = Parallel(n_jobs=8, verbose=True)

lst_df = paralelo(delayed(extraer)(url) for url in tqdm(urls[279:]))

  0%|          | 0/31 [00:00<?, ?it/s]

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


CPU times: total: 62.5 ms
Wall time: 1min 23s


[Parallel(n_jobs=8)]: Done  31 out of  31 | elapsed:  1.4min finished


In [108]:

pd.set_option('display.max_rows', None)
metar_october_2023 = pd.concat(lst_df)
# df[df.Condition == "Sunday"]
print(len(metar_october_2023))

1513


In [110]:
metar_october_2023.to_csv("../metar/metar_october_2023.csv", index=False)

In [117]:
meses2 = ["november","december"]
dias2 = []
for i in range(1,32):
    dias2.append(str(i))


In [118]:
urls22 = []
for mes in meses2:
    for dia in dias2:
        urls22.append(f'https://en.tutiempo.net/records/lemd/{dia}-{mes}-2022.html')

In [128]:
urls22[31]

'https://en.tutiempo.net/records/lemd/1-december-2022.html'

In [129]:
%%time
paralelo = Parallel(n_jobs=8, verbose=True)

lst_df = paralelo(delayed(extraer)(url) for url in tqdm(urls22[31:]))

  0%|          | 0/31 [00:00<?, ?it/s]

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.


CPU times: total: 78.1 ms
Wall time: 1min 41s


[Parallel(n_jobs=8)]: Done  31 out of  31 | elapsed:  1.7min finished


In [130]:
pd.set_option('display.max_rows', None)
metar_december_2022 = pd.concat(lst_df)
# df[df.Condition == "Sunday"]
print(len(metar_december_2022))

1491


In [131]:
metar_december_2022

,Day,Hour,Condition,Temperature,Wind,Relative_hum,Pressure
0,Thursday 1 December 2022,00:00,Clear,7°,4,87%,1015 hPa
1,Thursday 1 December 2022,00:30,Clear,7°,4,81%,1015 hPa
2,Thursday 1 December 2022,01:00,Clear,7°,4,87%,1015 hPa
3,Thursday 1 December 2022,01:30,Clear,7°,6,87%,1015 hPa
4,Thursday 1 December 2022,02:00,Clear,7°,4,87%,1015 hPa
5,Thursday 1 December 2022,02:30,Clear,7°,4,87%,1015 hPa
6,Thursday 1 December 2022,03:00,Clear,7°,4,87%,1015 hPa
7,Thursday 1 December 2022,03:30,Clear,6°,4,87%,1015 hPa
8,Thursday 1 December 2022,04:00,Clear,6°,4,87%,1014 hPa
9,Thursday 1 December 2022,04:30,Clear,4°,4,93%,1014 hPa


In [132]:
metar_december_2022.to_csv("../metar/metar_december_2022.csv", index=False)